### Imports

In [1]:
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from itertools import combinations
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bert_embedding import BertEmbedding
from allennlp.commands.elmo import ElmoEmbedder
from sklearn.model_selection import train_test_split

from transformers import *
import torch
import keras

import imp, gzip
import pickle, nltk
import gensim
import multiprocessing
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils as my_utils



import pandas as pd 
import numpy as np
import re
import collections
import matplotlib.pyplot as plt
from pathlib import Path

# Packages for data preparation
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

# Packages for modeling
from keras import models
from keras import layers
from keras import regularizers

Using TensorFlow backend.


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

### Definitions

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in tqdm(parse(path)):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [4]:
def process_df(df):
    df['text'] = my_utils.preprocess(df['text'])
    return df

### Start

In [ ]:
dataset_movies = getDF('datasets_raw/reviews_Movies_and_TV_5.json.gz')
dataset_movies.shape

In [5]:
dataset_kindle = getDF('datasets_raw/reviews_Kindle_Store_5.json.gz')
dataset_kindle.shape

982619it [01:28, 11154.18it/s]


(982619, 9)

In [ ]:
dataset_home = getDF('datasets_raw/reviews_Home_and_Kitchen_5.json.gz')
dataset_home.shape

In [6]:
dataset = dataset_kindle

In [ ]:
dataset = pd.concat([dataset_movies, dataset_home, dataset_kindle])

In [7]:
dataset = dataset.drop(columns=['reviewerID', 'asin', 'reviewerName', 'helpful', 'summary', 'unixReviewTime', 'reviewTime'])

In [8]:
dataset = dataset.rename(columns={'reviewText': 'text', 'overall': 'sentiment'})

In [9]:
dataset.shape

(982619, 2)

In [10]:
n_cores = 45

In [11]:
%%time
n = int(dataset.shape[0]/n_cores)
list_df = [dataset[i:i+n] for i in range(0, dataset.shape[0],n)]

pool = multiprocessing.Pool(n_cores)
processed_list_df = pool.map(process_df, list_df)
pool.close()

dataset = pd.concat(processed_list_df)
dataset.shape

CPU times: user 2.68 s, sys: 12.4 s, total: 15 s
Wall time: 4min 42s


(982619, 2)

In [12]:
Counter(dataset.sentiment)

Counter({5.0: 575264, 4.0: 254013, 3.0: 96194, 2.0: 34130, 1.0: 23018})

In [13]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in tqdm(f):
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [14]:
glove_embedding_dim = 300
glove_embeddings_index = loadGloveModel("nongit_resources/glove.6B.300d.txt")

760it [00:00, 7599.43it/s]

Loading Glove Model


400000it [01:04, 6205.27it/s]

Done. 400000  words loaded!


In [15]:
max_features = 20000

In [16]:
# dataset.text = dataset.text.apply(lambda x: " ".join(x))

In [17]:
# d = dataset.sample(1000000)

In [18]:
vectorizer = TfidfVectorizer(max_features = max_features)

In [19]:
vectorizer.fit(dataset.text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=20000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [20]:
vocab = vectorizer.get_feature_names()

In [ ]:
d = dataset.sample(300000)

In [ ]:
d.text = d.text.apply(lambda x: x.split(" "))

In [ ]:
# %%time
# d.text = d.text.apply(lambda x: [i for i in x if i in vocab])

In [ ]:
n_cores = 45

In [ ]:
def process_df(df):
    df['text'] = df['text'].apply(lambda x: [i for i in x if i in vocab])
    return df

In [ ]:
%%time
n = int(d.shape[0]/n_cores)
list_df = [d[i:i+n] for i in range(0, d.shape[0],n)]

pool = multiprocessing.Pool(n_cores)
processed_list_df = pool.map(process_df, list_df)
pool.close()

d_ = pd.concat(processed_list_df)
d_.shape

In [ ]:
d_ = d_[d_.text.apply(lambda x: len(x)>3 and len(x)<50)]

In [ ]:
d_.shape

In [ ]:
d_.text = d_.text.apply(lambda x: " ".join(x))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(d_.text, d_.sentiment, test_size=0.3, random_state=37)

In [ ]:
%%time
tk = Tokenizer(num_words=max_features, split=" ")
tk.fit_on_texts(X_train)

X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

In [ ]:
seq_lengths = X_train.apply(lambda x: len(x.split(' ')))
seq_lengths.describe()

In [ ]:
MAX_LEN = 50

In [ ]:
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_test_seq_trunc = pad_sequences(X_test_seq, maxlen=MAX_LEN)

In [ ]:
X_test_seq_trunc.shape

In [ ]:
X_train_seq_trunc.shape

In [ ]:
max_features

In [ ]:
len(tk.word_index.items())

In [ ]:
emb_matrix = np.zeros((max_features, glove_embedding_dim))

for w, i in tk.word_index.items():
    try:
        vect = glove_embeddings_index[w]
        emb_matrix[i] = vect
    except:
        pass

In [ ]:
emb_matrix.shape

In [ ]:
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)

In [ ]:
emb_model2 = models.Sequential()
emb_model2.add(layers.Embedding(max_features, 300, input_length=MAX_LEN))
emb_model2.add(Bidirectional(LSTM(1024)))
emb_model2.add(Dropout(0.7))
emb_model2.add(layers.Dense(256, activation='relu'))
emb_model2.add(layers.Dense(5, activation='softmax'))
emb_model2.summary()

In [ ]:
emb_model2.layers[0].set_weights([emb_matrix])
emb_model2.layers[0].trainable = True

In [ ]:
emb_model2.compile(optimizer='rmsprop', loss='categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
history = emb_model2.fit(X_train_seq_trunc , y_train_oh, epochs=10,
                    batch_size=512 , validation_data=(X_test_seq_trunc, y_test_oh), verbose=1)

In [ ]:
embeddings = emb_model2.layers[0].get_weights()[0]

In [ ]:
embeddings.shape

In [ ]:
len(tk.word_index.items())

In [ ]:
words_embeddings = {w:embeddings[idx] for w, idx in tk.word_index.items() if idx < max_features}

In [ ]:
import pickle

with open('nongit_resources/words_embeddings_trained2.pickle', 'wb') as handle:
    pickle.dump(words_embeddings, handle)